In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

## Finding transition weeks

### What is a transition week?
Transition weeks are the starting and ending week of a phenophase, for example, the first and last weeks of a tree's 'budding flowers' period. We can think of transition weeks as the first week that the percent of observations observing an attribute (say budding flowers) exceeds some small threshold, such as 10%.

### Our approach
We compute probability distributions for the start transition week and end transition week, $p_{start}$ and $p_{end}$, which range over all weeks in a year. We do this by normalizing score functions, $s_{start}$ and $s_{end}$, which are computed with respect to the plots of the percentage of observations each week of a species which observe a particular attribute.

### Start transition score function
$s_{start}(w)$ is large for weeks $w$ which are likely to be a start transition week. Let $y(w)$ be the percentage of observations for a species which observe the attribute in week $w$. Then:   
$$
    s_{start}(w) = w_1 \left(\frac{1}{0.1 + \left|\max_{w - L \leq k \leq w} y(k) - \min_{w - L \leq k \leq w} y(k)\right|}\right)^2 + w_2 (\max\left(y(w+M) - y(w), 0\right))
$$
a weighted sum over two terms. The first term, which we refer to as stagnation, computes the difference between the maximum and minimum percentages from $L$ weeks before week $w$ through week $w$ and is large when this difference is small. The second term, which we refer to as spike, computes the change in percentage from week $w$ to week $w+M$. This term is large when there is a large increase in percentage from week $w$ to week $w+M$.

### End transition score function
$s_{end}(w)$ is large for weeks $w$ which are likely to be an end transition week. Let $y(w)$ be the percentage of observations for a species which observe the attribute in week $w$. Then:
$$
    s_{end}(w) = w_1 \left(\frac{1}{0.1 + \left|\max_{w \leq k \leq w+L} y(k) - \min_{w \leq k \leq w+L} y(k)\right|}\right)^2 + w_2 (\max\left(y(w-M) - y(w), 0\right))
$$
a weighted sum over two terms. In this case, the stagnation term, computes the difference between the maximum and minimum percentages from week $w$ through week $w+L$ and is large when this difference is small. The second term, which we refer to as spike, computes the change in percentage from week $w-M$ to week $w$. This term is large when there is a large decrease in percentage from week $w-M$ to week $w$.

### Computing the probability distributions
We compute the distribution $p_{start}$, the probability that a given week is the start transition week, by normalizing the score function $s_{start}$.

$$
    p_{start}(w) = \frac{s_{start}(w)}{\sum_{w \in Y} s_{start}(w)}
$$

We compute the distribution $p_{end}$, the probability that a given week is the start transition week, by normalizing the score function $s_{end}$.
$$
    p_{end}(w) = \frac{s_{end}(w)}{\sum_{w \in Y} s_{end}(w)}
$$

### Computing Means and Standard Deviations
Means and standard deviations are computed using the same process for $p_{start}$ and $p_{end}$. Let $w_{a}$ be a start week. We compute the mean start transition time $\mu_{start}$ as follows:
$$
    \mu_{start} = \sum_{w = w_a}^{w_a + 48}w \cdot p_{start}(w)
$$
We compute the standard deviation of $p_{start}$, $\sigma_{start}$ as follows:
$$
    \sigma_{start} = \sqrt{\sum_{w = w_a}^{w_a + 48}(w - \mu_{start})^2 \cdot p_{start}(w)}
$$
In practice, we center our 48 week window over which means and standard deviations are computed around local maxima in the score function. So $w_a$ ends up being 24 weeks before each local maximum.

### Tuning Model Parameters $w_1, w_2, L, M$
Parameters to tune are $L$ (stagnation window size), $M$ (spike window size), $w_1$ (stagnation weight), and $w_2$ (spike weight). Our best practice currently is to set $w_2=1$, $w_1 = 0.25$, which reduces the amount stagnation factors into the score, ensuring that flat periods at the peaks of the percentage plot do not get high scores. We set $L=5$, which we found to be a reasonable number of weeks to look check back to identify periods of little change in the percentage plot of an attribute for a species. Most significantly, we set $M=15$, which we found to be a reasonable estimate of the number of weeks between the transition week and the peak week of the phenophase.

## Helper functions / data structures

In [2]:
# load in name_to_id lookup table
name_to_id_df = pd.read_csv('../data/species codes.csv', encoding='unicode_escape')
id_to_name_dict = {}
for _, row in name_to_id_df.iterrows():
    name = "{}-{}".format(row['species_primary_common_name'], row['species_scientific_name']).lower().replace(' ', '')
    id_to_name_dict[row['species_id']] = name

In [3]:
'''
Helper function to get the percent of observations of a species which observe an attribute for each week of a given year

Arguments:
    state_df (pandas.DataFrame) - dataframe of observations for a given state
    year (int) - year to compute percents for
    species_id (int) - id of the species to compute percents for
    attr (string) - attribute code. ex: 'Flowers_bud'

Returns:
    pcts (list) - a python list of percents of observations of a species which observe an attribute each week.
    
'''
def get_percent_of_positive_observations_for_each_week(state_df, year, species_id, attr):
    # filter state dataframe to get observations for just the species with id `species_id`
    species_df = state_df[state_df['Species_id'] == species_id]

    # filter species_df to only observations in the given year
    species_df = species_df[species_df['Year'] == year]

    # filter species_df to only rows where the attribute `attr` is observed to be 0 (none), 1 (some), 2 (many)
    species_df = species_df[species_df[attr] >= 0]

    # initialize the array `pcts`
    pcts = []
    for week in range(48):
        species_df_week = species_df[species_df['Week'] == week] # filter species_df to only data for the week
        N = len(species_df_week)
        if N == 0: # if this dataframe is empty, append 0 for percentage for that week
            pcts.append(0)
            #pcts.append(np.nan)
        else:
            species_df_week_observed = species_df_week[species_df_week[attr] > 0] # make dataframe of all observations for species which observe attr that week
            pcts.append(len(species_df_week_observed) / N) # compute percentage of observations of species that week which observe attr, store in array `pcts`
    return pcts # return the array of percents

## Score Function

In [4]:
'''
Computes scores proportional to the likelihood that a certain week is a transition week for a given attribute

Arguments:
    state_df - dataframe of all observations for a given state. example: all citizen observations with State_name 'Kerala'
    species_id - id of the species, an integer. example: 1161 (id of jackfruit)
    attr - the categorical attribute to get scores for. example: 'Fruits_ripe'

Parameters of Score function:
    L (integer) - size of stagnation window. Stagnation is computed as the difference between the maximum and minimum percentages of observations of the attribute `attr` 
        L weeks before (if phenophase_start=True) or after (if phenophase_start=False) the current week
    M (integer) - size of spike window. Spike is computed as the difference in percentage between the week M weeks after (if phenophase_start=True) or before (if phenophase_start=False) the current week, and the current week
    w_1 (float) - weight of the stagnation term in the score function
    w_2 (float) - weight of the spike term in the score function

Configuration Options
    phenophase_start (boolean) - If True, scores are computed for the transition at the start of the phenophase (allowing the distribution of transition times for the start transition week to be reconstructed). 
                                 If False, scores are computed for the transition at the end of the phenophase (allowing the distribution of transition times for the end transition week to be reconstructed).
'''
def get_scores(state_df, species_id, attr, L=5, M=15, w_1=0.25, w_2=1, phenophase_start=True):
    # compute percentage of observations of species `species_id` which observe attribute `attr` present for each week of the year, for 2018 through 2024
    pcts = np.array([])
    for year in range(2018, 2024):
        pcts = np.concatenate([pcts, get_percent_of_positive_observations_for_each_week(state_df, year, species_id, attr)], axis = 0)
    weeks = np.arange(48)
    
    # compute scores for start transition week distribution
    if phenophase_start:

        # compile arrays of scores, stagnations, and spikes for each week
        scores = []
        stagns = []
        spikes = []
        for week in range(L, len(pcts)-M, 1): # loop through all weeks of percents, except weeks where computing score would overflow through one end of the array `pcts`
            stagn = (1 / (0.1 + np.max(pcts[week-L:week+1]) - np.min(pcts[week-L:week+1])))**2  # compute stagnation term. Note that the 0.1 in denominator of `stagn` makes the 100 the maximum possible value of `stagn`.
            spike = max(0, pcts[week + M] - pcts[week]) # compute spike term
            spikes.append(spike)
            stagns.append(stagn)
        spikes = 100 * (np.array(spikes) - np.min(spikes)) / np.max(spikes) # normalize array of spikes to be between zero and 100
        scores = (w_1 * np.array(stagns) + w_2 * np.array(spikes)) / 2 # take score array to be average of stagnation and spike arrays

        # add zeros for padding to end of spikes, stagnations, and scores arrays to have a score for each week of the year
        spikes = np.array([0 for i in range(L)] + list(spikes) + [0 for i in range(M)]) 
        stagns = np.array([0 for i in range(L)] + list(stagns) + [0 for i in range(M)])
        scores = np.array([0 for i in range(L)] + list(scores) + [0 for i in range(M)])

        # return the score, spike, and stagnation arrays
        return scores, spikes, stagns

    # otherwise, compute scores for end transition week distribution
    else:
        # compile arrays of scores, stagnations, and spikes for each week
        scores = []
        stagns = []
        spikes = []
        for week in range(M, len(pcts)-L, 1):  # loop through all weeks of percents, except weeks where computing score would overflow through one end of the array `pcts`
            stagn = (1 / (0.1 + max(pcts[week:week+L+1]) - min(pcts[week:week+L+1])))**2 # compute stagnation term. Note that the 0.1 in denominator of `stagn` makes the 100 the maximum possible value of `stagn`
            spike = max(0, pcts[week - M] - pcts[week]) # compute spike term
            spikes.append(spike)
            stagns.append(stagn)
        spikes = 100 * (np.array(spikes) - np.min(spikes)) / np.max(spikes) # normalize array of spikes to be between zero and 100
        scores = (w_1 * np.array(stagns) + w_2 * np.array(spikes)) / 2 # take score array to be average of stagnation and spike arrays

        # add zeros for padding to end of spikes, stagnations, and scores arrays to have a score for each week of the year
        spikes = np.array([0 for i in range(L)] + list(spikes) + [0 for i in range(M)])
        stagns = np.array([0 for i in range(L)] + list(stagns) + [0 for i in range(M)])
        scores = np.array([0 for i in range(L)] + list(scores) + [0 for i in range(M)])

        # return the score, spike, and stagnation arrays
        return scores, spikes, stagns

## Compile and Plot Dataset of Mean Transition Times

In [5]:
'''
    Given a dataframe for a state, a species_id, and an attribute, compute the mean and standard deviation transition times for each year
    for that attribute. Return a dataframe with the information
'''
def get_means_and_stds(state_df, species_id, attr, L=5, M=3, w_1=1, w_2=1, w_3=1, state_name='Kerala'):
    scores_start, spikes_start, stagns_start = get_scores(state_df, species_id, attr, L=L, M=M, w_1=w_1, w_2=w_2,
                                                         phenophase_start=True)
    scores_end, spikes_end, stagns_end = get_scores(state_df, species_id, attr, L=L, M=M, w_1=w_1, w_2=w_2,
                                                         phenophase_start=False)
    start_means = []
    mean_start_probs = []
    end_means = []
    mean_end_probs = []
    max_prob_start_weeks = []
    max_start_probs = []
    max_prob_end_weeks = []
    max_end_probs = []

    for i in range(0,len(scores_start), 48):

        score_for_year = scores_start[range(i, i+48)].copy() # get scores for year
        
        max_start_score_idx = i+24 + np.argmax(scores_start[i+24:i+48]) # find max score in second half of year, will become 'center' of the start transition week distribution for year
        max_end_score_idx = i + np.argmax(scores_end[i:i+24]) # find max score in first half of year of year, will become 'center' of the end transition week distribution for year

        start_range = range(max(0, max_start_score_idx-24), min(len(scores_start), max_start_score_idx+24)) # compute week indices of the scores in the start distribution
        end_range = range(max(0, max_end_score_idx-24), min(len(scores_start), max_end_score_idx+24)) # compute week indices of scores in the ending distribution
   
        start_scores_for_year = scores_start[start_range].copy() # get all scores in window around center of start distribution
        end_scores_for_year = scores_end[end_range].copy() # get all scores in window around center of end distribution
        
        start_probs_for_year = start_scores_for_year / np.sum(start_scores_for_year) # normalize scores in start window to get probabilities
        end_probs_for_year = end_scores_for_year / np.sum(end_scores_for_year) # normalize scores in end window to get probabilities
    
        start_mean = np.dot(start_probs_for_year, start_range) # compute mean of start transition week distribution
        end_mean = np.dot(end_probs_for_year, end_range) # compute mean of ending transition week distribution
        start_means.append(round(start_mean % 48)) # add means to array, taking modulo 48 to convert to a week of the year, and rounding to nearest integer
        end_means.append(round(end_mean % 48)) 

        mean_start_probs.append((score_for_year / np.sum(start_scores_for_year))[round(start_mean % 48)])
        mean_end_probs.append((score_for_year / np.sum(end_scores_for_year))[round(end_mean % 48)])
    
        max_start_prob = np.max(start_probs_for_year) # compute maximum probability start week
        max_start_probs.append(max_start_prob)
        max_end_prob = np.max(end_probs_for_year) # compute maximum probability end week    
        max_end_probs.append(max_end_prob)  


        max_start_week = round(max_start_score_idx % 48) # compute maximum probability start week, round to nearest integer
        max_end_week = round(max_end_score_idx % 48) # compute maximum probability end week, round to nearest integer
        max_prob_start_weeks.append(max_start_week)
        max_prob_end_weeks.append(max_end_week)

    """
    Colum name | Description

    Species name | name of the species 

    Year | year of observation

    Phenophase | plant stage of observation

    Phenophase start week | week with maximum probability for initiation

    Max Initiation probability | max probability value for initiation

    Mean initiation week | week with average probability for initiation

    Mean initiation probability | mean probability value of initiation

    Phenophase end week | week with maximum probability for end

    Max end probability | max probability value for end

    Mean end week | week with average probability for end

    Mean end probability | mean probability value for end
    """

    # Create an empty dataframe to store transition times
    transition_time_df = pd.DataFrame(columns=['Species_name', 'Year', 'Phenophase', 'Phenophase_start_week',
                               'Max_Initiation_probability', 'Mean_initiation_week', 'Mean_initiation_probability', 'Phenophase_end_week', 'Max_end_probability', 'Mean_end_week', 'Mean_end_probability'])

    # get species and state name
    species_name = state_df[state_df['Species_id'] == species_id].iloc[0]['Species_name']
    
    # add all mean transition times and stds to dataframe and return the filled dataframe
    # one row for each year
    years = list(range(2018, 2024))
    for start_mean_prob, end_mean_prob, start_mean, end_mean, start_prob, end_prob, max_start, max_end, year in zip(mean_start_probs, mean_end_probs, start_means, end_means, max_start_probs, max_end_probs, max_prob_start_weeks, max_prob_end_weeks, years):
        transition_time_df.loc[len(transition_time_df)] = {'Species_name': species_name,'Year': year, 'Phenophase': attr, 
                                                           'Phenophase_start_week': max_start, 'Max_Initiation_probability': start_prob,
                                                           'Mean_initiation_week': start_mean, 'Mean_initiation_probability': start_mean_prob, 
                                                          'Phenophase_end_week': max_end, 'Max_end_probability': end_prob,
                                                          'Mean_end_week': end_mean, 'Mean_end_probability': end_mean_prob}
    # print(transition_time_df)
    return transition_time_df

In [6]:
# initialize dictionary which maps species id to a list of all attributes for which we will compute mean transition times for that species id 
transition_dict = {1090: ['Flowers_bud', 'Flowers_open', 'Fruits_unripe', 'Fruits_ripe'],
                   1161: ['Flowers_bud', 'Flowers_male', 'Flowers_Female', 'Fruits_unripe', 'Fruits_ripe']}

# load all kerala observations
kerala_df = pd.read_csv('../data/citizen_states_cleaned/kerala.csv')

# get a list of all transition dataframes, for each species_id / attr pair in transition_dict
transition_dfs = []
transition_time_df_mango_jack = []
for species_id in list(transition_dict.keys()):
    for attr in transition_dict[species_id]:
        transition_dfs.append(get_means_and_stds(kerala_df, species_id, attr, M=15, L=5, w_1=0.25, w_2=1)) # add transition dataframe for species / attr to the list

# join all dataframes together
transition_time_df_mango_jack = pd.concat(transition_dfs, ignore_index=True)
transition_time_df_mango_jack.to_csv("../data/year_to_year_transition_time.csv", index=False)

In [7]:
transition_time_df_mango_jack

,Species_name,Year,Phenophase,Phenophase_start_week,Max_Initiation_probability,Mean_initiation_week,Mean_initiation_probability,Phenophase_end_week,Max_end_probability,Mean_end_week,Mean_end_probability
0,Mango (all varieties)-Mangifera indica,2018,Flowers_bud,37,0.080885,32,0.042795,9,0.078031,14,0.001249
1,Mango (all varieties)-Mangifera indica,2019,Flowers_bud,33,0.083946,32,0.061640,11,0.087274,12,0.001240
2,Mango (all varieties)-Mangifera indica,2020,Flowers_bud,33,0.074016,30,0.063177,8,0.073567,8,0.000944
3,Mango (all varieties)-Mangifera indica,2021,Flowers_bud,35,0.071315,37,0.070194,9,0.063489,9,0.001954
4,Mango (all varieties)-Mangifera indica,2022,Flowers_bud,34,0.064178,32,0.056300,11,0.068561,11,0.003461
5,Mango (all varieties)-Mangifera indica,2023,Flowers_bud,27,0.135528,20,0.017784,8,0.074932,9,0.002084
6,Mango (all varieties)-Mangifera indica,2018,Flowers_open,33,0.078766,32,0.043086,9,0.073493,14,0.001177
7,Mango (all varieties)-Mangifera indica,2019,Flowers_open,36,0.081853,34,0.076870,11,0.084680,13,0.000676
8,Mango (all varieties)-Mangifera indica,2020,Flowers_open,33,0.071209,31,0.059765,8,0.073729,9,0.004183
9,Mango (all varieties)-Mangifera indica,2021,Flowers_open,37,0.066565,38,0.064819,7,0.063056,10,0.001382
